In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
import pyautogui
import time
import pandas as pd

In [2]:
#Nome do arquivo de entrada no formato CSV
arquivoEntrada = "pontos.csv"

#Nome do arquivo de saida no formato CSV
arquivoSaida = "pontosConvertidos.csv"

#Nomes das colunas que definem Latitude e Longitude
colunaLATITUDE = "Latitude"
colunaLONGITUDE = "Longitude"

'''
Selecionar o tipo de coordenada de entrada, por padrão o tipo de coordenada é Grau Decimal.
Abaixo temos os demais valores para selecionar caso necessário:
latlong_gd = GEOGRAFICA (Grau Decimal)
latlong_gms = GEOGRAFICA (Grau Minuto Segundo)
utm = UTM (metros)
policonica = POLICONICA (metros)
lambert = LAMBERT (metros)
mercator = MERCATOR (metros)
albers = ALBERS (metros)
cilequi = CILIN. EQUIDISTANTE (metros)
miller = MILLER (metros)
gaus = GAUSS (metros)

'''
cordEntrada = "latlong_gd"



'''
Selecionar o tipo de coordenada de saida, por padrão o tipo de coordenada é UTM.
Abaixo temos os demais valores para selecionar caso necessário:
utm = UTM (metros)
latlong = GEOGRAFICA
policonica = POLICONICA (metros)
lambert = LAMBERT (metros)
mercator = MERCATOR (metros)
albers = ALBERS (metros)
cilequi = CILIN. EQUIDISTANTE (metros)
miller = MILLER (metros)
gaus = GAUSS (metros)

'''
cordSaida = "utm"



'''
Selecionar os datums utilizado pelos dados de entrada e de saida, 
por padrão o datum de entrada é o WGS84 e o de saida é o SIRGAS2000.
Abaixo temos os demais valores para selecionar, caso necessário: 
1 = SAD 69
2 = CorregoAlegre
3 = Astro-Chua
4 = WGS84
5 = SIRGAS2000
'''
datumEntrada = '4'


datumSaida = '5'


In [3]:
# Iniciarliza o navegador e o site
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('http://www.dpi.inpe.br/calcula/')
driver.maximize_window()

driver.switch_to.default_content()
frame1 = driver.find_element(By.XPATH,'/html/frameset/frameset/frame')
frame2 = driver.find_element(By.XPATH,'/html/frameset/frameset/frameset/frame')
frame3 = driver.find_element(By.XPATH,'/html/frameset/frameset/frameset/frameset/frame')

# Inicializa o arquivo de entrada
file = pd.read_csv(arquivoEntrada, ';')
#display(file)

#Faz uma cópia do arquivo de entrada pra uma tabela, caso haja algum erro
tabela = file

tabela["LAT_UTM"] = " "
tabela["LON_UTM"] = " "
tabela["ZONA_UTM"] = " "

size = int(tabela[colunaLATITUDE].size)

for i in range(0, size):
        
        x = str(tabela[colunaLONGITUDE].loc[i])
        y = str(tabela[colunaLATITUDE].loc[i])

        x = x.replace(",",".")
        y = y.replace(",",".")


        '''print(x)
        print(y)
        print(type(x))'''


        driver.switch_to.default_content()
        driver.switch_to.frame(frame1)

        # Selecionar tipo de coordenada de entrada
        Select(driver.find_element(By.XPATH, '/html/body/div[1]/center/table/tbody/tr[3]/td/select')).select_by_value(cordEntrada)

        # Seleciona datum de entrada
        Select(driver.find_element(By.XPATH, '/html/body/div[1]/center/table/tbody/tr[9]/td/select')).select_by_value(datumEntrada)
        
        driver.find_element(By.XPATH,'/html/body/div[1]/center/table/tbody/tr[5]/td/input[1]').clear()
        driver.find_element(By.XPATH,'/html/body/div[1]/center/table/tbody/tr[7]/td/input[1]').clear()


        driver.find_element(By.XPATH,'/html/body/div[1]/center/table/tbody/tr[5]/td/input[1]').send_keys(x)
        driver.find_element(By.XPATH,'/html/body/div[1]/center/table/tbody/tr[7]/td/input[1]').send_keys(y)
        driver.find_element(By.XPATH, '/html/body/div[1]/center/table/tbody/tr[10]/td/input').click()

        driver.switch_to.default_content()
        driver.switch_to.frame(frame2)

        #Selecionar tipo de coordenada de saida
        Select(driver.find_element(By.XPATH, '/html/body/table/tbody/tr[2]/td/select')).select_by_value(cordSaida)

        # Selecionar datum de saida
        Select(driver.find_element(By.XPATH, '/html/body/table/tbody/tr[4]/td/select')).select_by_value(datumSaida)

        driver.find_element(By.XPATH, '/html/body/table/tbody/tr[5]/td/input').click()

        driver.switch_to.default_content()
        driver.switch_to.frame(frame3)

        latUTM = driver.find_element(By.XPATH,'/html/body/table[1]/tbody/tr[8]/td[2]/font/i').get_attribute('innerHTML')
        #print("Latitude UTM: ", latUTM)

        lonUTM = driver.find_element(By.XPATH, '/html/body/table[1]/tbody/tr[12]/td[2]/font/i').get_attribute('innerHTML')
        #print("Longitude UTM: ",lonUTM)

        zonaUTM = driver.find_element(By.XPATH, '/html/body/table[2]/tbody/tr/td/font/i').get_attribute('innerHTML')
        #print(zonaUTM)

        
        latUTM = str(latUTM).replace(".",",")
        lonUTM = str(lonUTM).replace(".",",")
        
        latUTM = latUTM.replace(" ", "")
        lonUTM = lonUTM.replace(" ", "")
        
        zonaUTM = zonaUTM[39:41].replace(" ","")
        
        #latUTM.replace(".",",")
        #lonUTM.replace(".",",")
        
        tabela["LAT_UTM"].loc[i] = latUTM
        tabela["LON_UTM"].loc[i] = lonUTM
        tabela["ZONA_UTM"].loc[i] = zonaUTM


#display(tabela)
tabela.to_csv(arquivoSaida,";", index= False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\user\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
C:\Users\user\AppData\Local\Temp\ipykernel_2608\2830363075.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\user\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3361: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\user\AppData\Local\Temp\ipykernel_2608\2830363075.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan